<a href="https://colab.research.google.com/github/ykitaguchi77/Kaggle/blob/main/Kaggle_setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Kaggle on Google Colab**

In [1]:
#まず設定をする
class Config:
    name = "baseline-001"
    only_inference = False #学習なし推論のみの場合にTrue

    model_name = "roberta-base"
    learning_rate = 1e-5
    max_length = 256
    epochs = 8
    batch_size = 16

    n_fold = 5
    trn_fold = [0, 1, 2, 3, 4]
    seed = 2022
    target_col = "target"
    debug = False

    # Colab Env
    upload_from_colab = True
    api_path = "/content/drive/MyDrive/kaggle.json"
    drive_path = "/content/drive/MyDrive/CommonLit-Readability-Prize"
    
    # Kaggle Env
    kaggle_dataset_path = None

In [2]:
import os
import json
import warnings
import shutil
import logging
import joblib
import random
import datetime
import sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm.auto import tqdm
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

import tensorflow as tf
from tensorflow.keras import backend as K

#**Import utils** 

In [4]:
class Logger:
    """参考) https://github.com/ghmagazine/kagglebook/blob/master/ch04-model-interface/code/util.py"""
    def __init__(self, path):
        self.general_logger = logging.getLogger(path)
        stream_handler = logging.StreamHandler()
        file_general_handler = logging.FileHandler(os.path.join(path, 'Experiment.log'))
        if len(self.general_logger.handlers) == 0:
            self.general_logger.addHandler(stream_handler)
            self.general_logger.addHandler(file_general_handler)
            self.general_logger.setLevel(logging.INFO)

    def info(self, message):
        # display time
        self.general_logger.info('[{}] - {}'.format(self.now_string(), message))

    @staticmethod
    def now_string():
        return str(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

def seed_everything(seed=42):
    #参考) https://qiita.com/kaggle_grandmaster-arai-san/items/d59b2fb7142ec7e270a5
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    

#**Setup**

In [7]:
!pip install kaggle
import os
import json

f = open("/content/drive/MyDrive/Kaggle/kaggle.json", 'r')
json_data = json.load(f) 
os.environ['KAGGLE_USERNAME'] = json_data['username']
os.environ['KAGGLE_KEY'] = json_data['key']

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
#データのダウンロード
#コンペティションのサイト --> Data --> Kaggle APIをコピぺ

!kaggle competitions download -c rsna-2022-cervical-spine-fracture-detection

  5% 10.3G/189G [02:03<35:44, 89.7MB/s]
User cancelled operation
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.7/dist-packages/kaggle/cli.py", line 78, in main
    out = None
KeyboardInterrupt


In [ ]:
import shutil
new_path = shutil.move('temp/dir2/dir/', 'temp/dir1/dir_new/')


In [5]:
COLAB = "google.colab" in sys.modules

if COLAB:
    print("This environment is Google Colab")
    
    # mount
    from google.colab import drive
    if not os.path.isdir("/content/drive"):
        drive.mount('/content/drive') 
	
    # import library
    ! pip install --quiet transformers
    ! pip install --quiet iterative-stratification
    ! pip install --quiet tensorflow-addons

    # use kaggle api (need kaggle token)
    f = open(Config.api_path, 'r')
    json_data = json.load(f) 
    os.environ["KAGGLE_USERNAME"] = json_data["username"]
    os.environ["KAGGLE_KEY"] = json_data["key"]
    
    # set dirs
    DRIVE = Config.drive_path
    EXP = (Config.name if Config.name is not None 
           else get("http://172.28.0.2:9000/api/sessions").json()[0]["name"][:-6])
    INPUT = os.path.join(DRIVE, "Input")
    OUTPUT = os.path.join(DRIVE, "Output")
    SUBMISSION = os.path.join(DRIVE, "Submission")
    OUTPUT_EXP = os.path.join(OUTPUT, EXP) 
    EXP_MODEL = os.path.join(OUTPUT_EXP, "model")
    EXP_FIG = os.path.join(OUTPUT_EXP, "fig")
    EXP_PREDS = os.path.join(OUTPUT_EXP, "preds")

    # make dirs
    for d in [INPUT, SUBMISSION, EXP_MODEL, EXP_FIG, EXP_PREDS]:
        os.makedirs(d, exist_ok=True)

    if not os.path.isfile(os.path.join(INPUT, "train.csv.zip")):
        # load dataset
        ! kaggle competitions download -c commonlitreadabilityprize -p $INPUT 
    
    # utils
    logger = Logger(OUTPUT_EXP)

This environment is Google Colab
Mounted at /content/drive
     |████████████████████████████████| 4.7 MB 7.7 MB/s 
     |████████████████████████████████| 101 kB 8.4 MB/s 
     |████████████████████████████████| 6.6 MB 62.9 MB/s 
     |████████████████████████████████| 596 kB 55.2 MB/s 
     |████████████████████████████████| 1.1 MB 6.6 MB/s 


FileNotFoundError: ignored